In [1]:
import tensorflow as tf

2023-07-29 21:18:10.346214: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.list_physical_devices()

2023-07-29 21:18:11.594816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 21:18:11.599047: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-29 21:18:11.599149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import os
import openai
import sys
import glob
import tiktoken
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

open.api_key = os.environ['OPENAI_API_KEY']

KeyError: 'OPENAI_API_KEY'

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [6]:
glob.glob("*.pdf")

['The_Effect_of_Student_Teacher_Ratio_on_Truancy.pdf',
 'Question_Generation.pdf',
 'fec_2016_EDA.v2.pdf',
 'w207-music-genre-FINAL-presentation.pdf',
 'exploring-ggplot.pdf']

In [7]:
loaders = [PyPDFLoader(pdf) for pdf in glob.glob("*.pdf")]

In [8]:
loaders

In [9]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [10]:
len(docs)

74

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [12]:
splits = text_splitter.split_documents(docs)

In [13]:
len(splits)

140

In [14]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [15]:
from langchain.vectorstores import Chroma

In [16]:
persist_directory = 'docs/chroma/'

In [17]:
!rm -rf ./docs/chroma  # remove old database files if any

In [18]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [19]:
print(vectordb._collection.count())

140


In [20]:
question = "How do lexical metrics compare to semantic metrics for NLP text generation evaluation?"

In [21]:
docs = vectordb.similarity_search(question,k=10)

In [22]:
len(docs)

10

In [23]:
docs[2].page_content

'be true for our question generation models. That is why\nwe introduced the blended dataset described in Section\n3.\nWe also believed that shuffling the data in the blended\ndataset would improve model performance by introduc-\ning more variation into the training process and avoiding\npotential local minima reflective of consistent patterns\nfound in the source datasets.\n4.3 Metrics\nWe chose evaluation metrics currently popular in evalu-\nating sequence-to-sequence text generation tasks. These\nmetrics evaluate the alignment of textual features such\nas word choice, word order, or semantics between refer-\nence and candidate sequences. We selected metrics that\nmeasure similarity in different ways to explore which\nfeatures contribute most to faithfulness, fluidity, and\ncoherence.\nOur metrics fall into two categories: measures of\nlexical similarity and measures of semantic similarity.\nWe use BLEU, ROUGE-L, and METEOR to evaluate\nthe lexical similarity between pairs of generate

In [24]:
vectordb.persist()

In [25]:
for doc in docs:
    print(doc.metadata)

{'page': 3, 'source': 'Question_Generation.pdf'}
{'page': 13, 'source': 'Question_Generation.pdf'}
{'page': 1, 'source': 'Question_Generation.pdf'}
{'page': 14, 'source': 'Question_Generation.pdf'}
{'page': 3, 'source': 'Question_Generation.pdf'}
{'page': 2, 'source': 'Question_Generation.pdf'}
{'page': 0, 'source': 'Question_Generation.pdf'}
{'page': 0, 'source': 'Question_Generation.pdf'}
{'page': 2, 'source': 'Question_Generation.pdf'}
{'page': 4, 'source': 'Question_Generation.pdf'}
